### 数据导入

In [ ]:
import pandas as pd
import numpy as np
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RepeatedKFold, cross_val_score, cross_val_predict, KFold
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler, StandardScaler

In [ ]:
import pandas as pd
train_data = pd.read_excel('/content/drive/MyDrive/BuildModel/Molecular_Descriptor.xlsx')#,sheet_name='test')
train_label = pd.read_excel('/content/drive/MyDrive/BuildModel/ADMET.xlsx')#,sheet_name='test')
# 获取train_data的描述性数据
t_describe = train_data.describe()
t_describe = pd.DataFrame(t_describe.values.T, index=t_describe.columns, columns=t_describe.index)
drop_index = pd.Series(t_describe[t_describe['mean']==0.].index)
train_data = train_data.drop(drop_index,axis=1)
train_data = train_data.set_index('SMILES')
train_data.head()

,nAcid,ALogP,ALogp2,AMR,apol,naAromAtom,nAromBond,nAtom,nHeavyAtom,nH,nC,nN,nO,nS,nP,nF,nCl,nBr,nI,nX,ATSc1,ATSc2,ATSc3,ATSc4,ATSc5,ATSm1,ATSm2,ATSm3,ATSm4,ATSm5,ATSp1,ATSp2,ATSp3,ATSp4,ATSp5,nBase,BCUTw-1l,BCUTw-1h,BCUTc-1l,BCUTc-1h,...,n6Ring,n7Ring,n8Ring,n12Ring,nG12Ring,nFRing,nF6Ring,nF7Ring,nF8Ring,nF9Ring,nF10Ring,nF11Ring,nF12Ring,nFG12Ring,nTRing,nT4Ring,nT5Ring,nT6Ring,nT7Ring,nT8Ring,nT9Ring,nT10Ring,nT11Ring,nT12Ring,nTG12Ring,nRotB,LipinskiFailures,TopoPSA,VABC,VAdjMat,MW,WTPT-1,WTPT-2,WTPT-3,WTPT-4,WTPT-5,WPATH,WPOL,XLogP,Zagreb
SMILES,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCC3)c4ccc(OCCN5CCCCC5)cc4,0,-0.2860,0.081796,126.1188,74.170169,12,12,64,31,33,26,1,3,1,0,0,0,0,0,0,0.460175,-0.217040,-0.126453,0.161524,-0.080148,39.811140,40.498576,58.166514,61.613797,58.060335,2469.977567,2930.668527,4109.492845,4128.339568,3792.779760,1,11.85,31.972073,-0.360525,0.185437,...,4,0,0,0,0,1,0,0,0,0,1,0,0,0,6,0,1,4,0,0,0,1,0,0,0,6,0,67.23,414.727478,5.954196,439.218115,64.771680,2.089409,15.471445,8.858910,3.406628,3011,47,4.666,166
Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCCCC3)c4ccc(OCCN5CCCCC5)cc4,0,-0.8620,0.743044,131.9420,80.357341,12,12,70,33,37,28,1,3,1,0,0,0,0,0,0,0.460175,-0.217040,-0.126453,0.161524,-0.080148,41.811140,42.498576,60.166514,68.613797,60.060335,2679.488262,3157.289284,4379.505354,4811.560374,4154.384741,1,11.85,31.972073,-0.360530,0.185459,...,4,1,0,0,0,1,0,0,0,0,1,0,0,0,6,0,0,4,1,0,0,1,0,0,0,6,1,67.23,449.319447,6.044394,467.249415,68.960024,2.089698,15.486947,8.863774,3.406648,3516,54,5.804,174
Oc1ccc(cc1)[C@H]2Sc3cc(O)ccc3O[C@H]2c4ccc(OCCN5CCCCC5)cc4,0,0.7296,0.532316,139.9304,74.064997,18,18,62,33,29,27,1,4,1,0,0,0,0,0,0,0.519350,-0.236031,-0.143126,0.175159,-0.059065,42.585609,42.830668,61.830698,67.277981,60.392427,2576.569862,3054.050592,4285.592752,4479.177597,3993.734054,1,11.85,31.972073,-0.361379,0.116954,...,5,0,0,0,0,1,0,0,0,0,1,0,0,0,6,0,0,5,0,0,0,1,0,0,0,6,0,87.46,422.004313,6.044394,463.181729,68.748923,2.083301,18.011114,11.390412,3.406644,3542,52,2.964,176
Oc1ccc2O[C@H]([C@@H](CC3CCCCC3)Sc2c1)c4ccc(OCCN5CCCCC5)cc4,0,-0.3184,0.101379,133.4822,80.357341,12,12,70,33,37,28,1,3,1,0,0,0,0,0,0,0.460206,-0.216970,-0.126504,0.161226,-0.079743,41.811140,42.498576,60.166514,62.944012,57.728243,2596.316352,3045.933534,4193.549346,4311.294610,3850.190106,1,11.85,31.972073,-0.360530,0.185459,...,5,0,0,0,0,1,0,0,0,0,1,0,0,0,6,0,0,5,0,0,0,1,0,0,0,7,1,67.23,449.319447,6.044394,467.249415,68.883696,2.087385,15.468365,8.857943,3.406624,3594,50,6.015,174
Oc1ccc2O[C@H]([C@@H](Cc3ccccc3)Sc2c1)c4ccc(OCCN5CCCCC5)cc4,0,1.3551,1.836296,143.1903,76.356583,18,18,64,33,31,28,1,3,1,0,0,0,0,0,0,0.462912,-0.215099,-0.128690,0.152869,-0.069664,41.811140,42.498576,60.166514,62.944012,57.728243,2561.845107,3007.231045,4144.672227,4265.064807,3806.120673,1,11.85,31.972073,-0.360530,0.185459,...,5,0,0,0,0,1,0,0,0,0,1,0,0,0,6,0,0,5,0,0,0,1,0,0,0,7,0,67.23,430.510071,6.044394,461.202465,68.883696,2.087385,15.468365,8.857943,3.406624,3594,50,4.462,174


#### 特征处理

In [ ]:
#attr_labels = pd.read_excel("/content/drive/MyDrive/BuildModel/pearsonr_labels.xlsx").loc[:,0]
#attr_labels = pd.read_excel("/content/drive/MyDrive/BuildModel/rf_labels.xlsx").loc[:,1]
# attr_labels = list(attr_labels)
# attr_labels

In [ ]:
# 特征选择
t_data = train_data
#t_data = train_data.loc[:,attr_labels]
print(t_data.head())
t_label = train_label.iloc[:,5]
print(t_label.head())
# 标准化
# std_scaler = StandardScaler().fit(t_data)
# t_scaledata = std_scaler.transform(t_data)

# 0-1区间缩放
mm_scaler = MinMaxScaler().fit(t_data)
t_scaledata = mm_scaler.transform(t_data)

                                                    nAcid  ...  Zagreb
SMILES                                                     ...        
Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCC3)c4ccc(OCCN5CC...      0  ...     166
Oc1ccc2O[C@H]([C@H](Sc2c1)C3CCCCCC3)c4ccc(OCCN5...      0  ...     174
Oc1ccc(cc1)[C@H]2Sc3cc(O)ccc3O[C@H]2c4ccc(OCCN5...      0  ...     176
Oc1ccc2O[C@H]([C@@H](CC3CCCCC3)Sc2c1)c4ccc(OCCN...      0  ...     174
Oc1ccc2O[C@H]([C@@H](Cc3ccccc3)Sc2c1)c4ccc(OCCN...      0  ...     174

[5 rows x 504 columns]
0    0
1    0
2    1
3    0
4    0
Name: MN, dtype: int64


In [ ]:
t_d.head()

#### 设置训练数据集

In [ ]:
# from sklearn.feature_selection import SelectFromModel
# from sklearn.ensemble import RandomForestClassifier

# t_scaledata = SelectFromModel(RandomForestClassifier(),threshold="0.75*mean").fit_transform(t_scaledata, t_label)

In [ ]:
t_scaledata.shape

(1974, 504)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(t_scaledata, t_label, test_size=0.1, random_state=0)

### 数据预处理

### 模型建立

#### 网格训练模型

In [ ]:
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

def train_model(model, param_grid=[], X_train=[], y_train=[], X_test=[], y_test=[]):

  gsearch = GridSearchCV(model, param_grid, cv=5, n_jobs=-1, scoring='precision')
  gsearch.fit(X_train, y_train)

  # 提取最好的模型
  model = gsearch.best_estimator_
  # 预测

  print("Best model:")
  print(model)
  print('----------------------')
  print('The score for trainset: ', model.score(X_train, y_train))

  print("Detailed testset report:")
  print('The score for testset: ', model.score(X_test, y_test))
  y_pred = model.predict(X_test)
  print(classification_report(y_test, y_pred))
  print("roc_auc_score of testset:",roc_auc_score(y_test, y_pred, multi_class='ovr'))

  return model

In [ ]:
opt_models = dict()

#### 单一模型预测

##### LR模型

In [ ]:
from sklearn.linear_model import LogisticRegression
model = 'LogisticRegression'
param_grid = {"max_iter":[500,550,600]}
opt_models[model] = train_model(LogisticRegression(), param_grid, X_train, y_train, X_test, y_test)

Best model:
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=500,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)
----------------------
The score for trainset:  0.9290540540540541
Detailed testset report:
The score for testset:  0.9343434343434344
              precision    recall  f1-score   support

           0       0.90      0.81      0.85        47
           1       0.94      0.97      0.96       151

    accuracy                           0.93       198
   macro avg       0.92      0.89      0.91       198
weighted avg       0.93      0.93      0.93       198

roc_auc_score of testset: 0.8910102860363533


##### KNN模型

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model = 'KNeighborsClassifier'
param_grid = {"n_neighbors":[5,6,7,8,9,10]}
opt_models[model] = train_model(KNeighborsClassifier(), param_grid, X_train, y_train, X_test, y_test)

Best model:
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                     weights='uniform')
----------------------
The score for trainset:  0.9391891891891891
Detailed testset report:
The score for testset:  0.9191919191919192
              precision    recall  f1-score   support

           0       0.83      0.83      0.83        47
           1       0.95      0.95      0.95       151

    accuracy                           0.92       198
   macro avg       0.89      0.89      0.89       198
weighted avg       0.92      0.92      0.92       198

roc_auc_score of testset: 0.888403550796111


##### 决策树模型

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model = 'DecisionTreeClassifier'
param_grid = {"max_depth":[5,7,9,11]}
opt_models[model] = train_model(DecisionTreeClassifier(), param_grid, X_train, y_train, X_test, y_test)

Best model:
DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=11, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')
----------------------
The score for trainset:  0.9960585585585585
Detailed testset report:
The score for testset:  0.9545454545454546
              precision    recall  f1-score   support

           0       0.91      0.89      0.90        47
           1       0.97      0.97      0.97       151

    accuracy                           0.95       198
   macro avg       0.94      0.93      0.94       198
weighted avg       0.95      0.95      0.95       198

roc_auc_score of testset: 0.933563477525715


#### 模型融合Boosting

##### ETtree

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

model = 'ExtraTreesClassifier'
param_grid = {"n_estimators":[100,150,200,250],'max_depth':[5,8,11,14]}
opt_models[model] = train_model(ExtraTreesClassifier(), param_grid, X_train, y_train, X_test, y_test)

Best model:
ExtraTreesClassifier(bootstrap=False, ccp_alpha=0.0, class_weight=None,
                     criterion='gini', max_depth=14, max_features='auto',
                     max_leaf_nodes=None, max_samples=None,
                     min_impurity_decrease=0.0, min_impurity_split=None,
                     min_samples_leaf=1, min_samples_split=2,
                     min_weight_fraction_leaf=0.0, n_estimators=250,
                     n_jobs=None, oob_score=False, random_state=None, verbose=0,
                     warm_start=False)
----------------------
The score for trainset:  1.0
Detailed testset report:
The score for testset:  0.9696969696969697
              precision    recall  f1-score   support

           0       1.00      0.87      0.93        47
           1       0.96      1.00      0.98       151

    accuracy                           0.97       198
   macro avg       0.98      0.94      0.96       198
weighted avg       0.97      0.97      0.97       198

roc_auc_sco

##### GBDT模型

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

model = 'GradientBoostingClassifier'
param_grid = {"n_estimators":[150,200,250],'max_depth':[5,7,9,11]}
opt_models[model] = train_model(GradientBoostingClassifier(), param_grid, X_train, y_train, X_test, y_test)

Best model:
GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=250,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)
----------------------
The score for trainset:  1.0
Detailed testset report:
The score for testset:  0.9797979797979798
              precision    recall  f1-score   support

           0       0.98      0.94      0.96        47
           1       0.98      0.99      0.99       151

    acc

##### 随机森林模型

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model = 'RandomForestClassifier'
param_grid = {"n_estimators":[100,150,200],'max_depth':[11,13,15]}
opt_models[model] = train_model(RandomForestClassifier(), param_grid, X_train, y_train, X_test, y_test)


Best model:
RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)
----------------------
The score for trainset:  1.0
Detailed testset report:
The score for testset:  0.9696969696969697
              precision    recall  f1-score   support

           0       0.98      0.89      0.93        47
           1       0.97      0.99      0.98       151

    accuracy                           0.97       198
   macro avg       0.97      0.94      0.96       198
weighted avg       0.97      0.97      0.97       1

##### Adaboosting

In [113]:
from sklearn.ensemble import AdaBoostClassifier

model = 'AdaBoostClassifier'
param_grid = {"n_estimators":[100,150,200,250]}
opt_models[model] = train_model(AdaBoostClassifier(), param_grid, X_train, y_train, X_test, y_test)

Best model:
AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=150, random_state=None)
----------------------
The score for trainset:  1.0
Detailed testset report:
The score for testset:  0.9646464646464646
              precision    recall  f1-score   support

           0       0.93      0.91      0.92        47
           1       0.97      0.98      0.98       151

    accuracy                           0.96       198
   macro avg       0.95      0.95      0.95       198
weighted avg       0.96      0.96      0.96       198

roc_auc_score of testset: 0.9475130336762012


#### 投票法

In [114]:
from sklearn.ensemble import VotingClassifier

models = [(m,opt_models[m]) for m in opt_models.keys() if m not in ["KNeighbors","LogisticRegression"]]
vot_clf = VotingClassifier(models).fit(X_train, y_train)
print("Detailed testset report:")
print('The score for testset: ', vot_clf.score(X_test, y_test))
y_pred = vot_clf.predict(X_test)
print(classification_report(y_test, y_pred))
print("roc_auc_score of testset:",roc_auc_score(y_test, y_pred, multi_class='ovr'))

Detailed testset report:
The score for testset:  0.9696969696969697
              precision    recall  f1-score   support

           0       0.98      0.89      0.93        47
           1       0.97      0.99      0.98       151

    accuracy                           0.97       198
   macro avg       0.97      0.94      0.96       198
weighted avg       0.97      0.97      0.97       198

roc_auc_score of testset: 0.943497252360152


####保存优秀的模型

In [115]:
from joblib import dump, load
dump(vot_clf, '/content/drive/MyDrive/BuildModel/question_3_5.joblib')
clf = load('/content/drive/MyDrive/BuildModel/question_3_5.joblib')
clf.score(X_test,y_test)

['/content/drive/MyDrive/BuildModel/question_3_5.joblib']

In [116]:
clf = load('/content/drive/MyDrive/BuildModel/question_3_5.joblib')
clf.score(X_test,y_test)

0.9696969696969697